In [1]:
import sqlite3 as sql
import math
import catalog_utils as cu

In [2]:
def init(con, source):
    cur = con.cursor()
    target = source + '_p3'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT 
                   desig, 
                   m3_6, m4_5, m5_8, m8_0, m24, 
                   c3624, c4524, c5824, c8024, c3658,
                   Type 
                   FROM '''+ source +''' WHERE
                   (m24 IS NOT NULL) AND
                   dm24 < 0.1 AND
                   (dm3_6 < 0.2 OR m3_6 IS NULL) AND
                   (dm4_5 < 0.2 OR m4_5 IS NULL) AND
                   (dm5_8 < 0.2 OR m5_8 IS NULL) AND
                   (dm8_0 < 0.2 OR m8_0 IS NULL)''')
    return target

In [3]:
def extract_3(con, source, cutoff36):
    #flag what were considered photospheric to transition disks
    cur = con.cursor()
    target = source + '_3'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT
                   desig FROM '''+ source +''' WHERE 
                   (Type = 1 OR 2)AND 
                   (c5824 > 2.5 OR c4524 > 2.5)
                   AND m3_6 < ?
                   ''', [cutoff36])
    return target

In [4]:
#Class I* 
def extract_18(con, source, cutoff24):
    cur = con.cursor()
    target = source + '_18'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                     SELECT desig FROM (SELECT desig, 
                      CASE
                      WHEN m8_0 IS NOT NULL THEN c8024
                      WHEN (m8_0 IS NULL) AND (m5_8 IS NOT NULL) THEN c5824
                      WHEN (m8_0 IS NULL) AND (m5_8 IS NULL) AND (m4_5 IS NOT NULL) THEN c4524
                      WHEN (m8_0 IS NULL) AND (m5_8 IS NULL) AND (m4_5 IS NULL) AND (m3_6 IS NOT NULL) THEN c3624
                      END AS cX_24
                     FROM ''' + source + ''' WHERE m24 < ?)
                    WHERE cX_24 > 4.5''', [cutoff24])
    return target

In [5]:
#reflag AGN and shock emissions as class I
def reflag_299_to_1(con, source, cutoff24):
    cur = con.cursor()
    target = source + '_1_from_29_9'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+ source +''' WHERE 
                   (Type = 29 OR Type = 9) AND 
                   m24 > ? AND
                   c3658 > 0.5 AND
                   c4524 > 4.5 AND
                   c8024 > 4''', [cutoff24])
    return target

In [6]:
#reflag previously flagged c1 to c2
def reflag_1_to_2(con, source):
    cur = con.cursor()
    target = source+'_2_from_1'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+source+''' WHERE 
                   Type = 1 AND 
                   (m5_8 IS NOT NULL AND c5824 <=4) OR
                   (m5_8 IS NULL AND c4524 <=4)''')
    return target

In [7]:
def run_phase3(master, constr, cutoff36, cutoff24):
    con = sql.connect(constr)
    cur = con.cursor()
    
    wc = init(con, master)
    cu.update_phase(con, master, wc, 0)
    
    c3 = extract_3(con, wc, cutoff36)
    cu.update_type(con, master, c3, 3)
    cu.update_type(con, wc, c3, 3)
    
    #extract ClassI*
    c18 = extract_18(con, wc, cutoff24)
    cu.update_type(con, master, c18, 0)
    cu.update_type(con, wc, c18, 0)
    
    #reflag AGN and shock emissions as classI
    c1f299 = reflag_299_to_1(con, wc, cutoff24)
    cu.update_type(con, master, c1f299, 1)
    cu.update_type(con, wc, c1f299, 1)
    
    #reflag previous class I to classII
    c2f1 = reflag_1_to_2(con, wc)
    cu.update_type(con, master, c2f1, 2)
    cu.update_type(con, wc, c2f1, 2)
    
    return 0

In [8]:
run_phase3('gtw49', '/home/kecai/w49/w49test.db', 15.2, 9.7)

0